In [7]:
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
import numpy as np

In [299]:
test_data_sample_cpy = pd.read_csv('test_data_sample_cpy_new.csv')

In [171]:
test_data_sample_cpy.head()

,title,paras,sents,words,author,pubdate,media,topic,num_media,num_topic,name_media,name_topic,new_para,cands
0,['Here are 11 climate change policies to fight...,['Last year\'s\xa0report\xa0from the Intergove...,['Last year\'s\xa0report\xa0from the Intergove...,"['Last', 'year', ""'"", 's', 'report', 'from', '...","[['Peter Buckland', 'Brandi Robinson', 'Durwoo...",['2019-01-02 09:00:53'],['Washington Post'],['welfare'],1,6,Liberal,Employment & Trade,"[['last', 'yield', 's', 'editor', 'from', 'the...","[{'sent_idx': 0, 'word_idx': [1, 13, 24, 54, 5..."
1,['How The Federal Shutdown Is Affecting Health...,['\n Despite the partial shutdo...,['\n Despite the partial shutdo...,"['Despite', 'the', 'partial', 'shutdown', ',',...",[['Shefali Luthra']],['2019-01-03 15:55:14'],['NPR'],['social media regulation'],1,3,Liberal,Miscellaneous,"[['amid', 'the', 'partial_shutdown', ',', 'the...","[{'sent_idx': 0, 'word_idx': [0, 8, 11], 'orig..."
2,['The Moneyist: I received $500 from an auto i...,"[' Dear Moneyist, ', ""Here's an ethical questi...","[' Dear Moneyist,', ""Here's an ethical questio...","['Dear', 'Moneyist', ',', 'Here', ""'"", 's', 'a...","[['Quentin Fottrell', 'Personal Finance Editor']]",['2019-01-05 12:22:12'],['Wall Street Journal'],['campaign finance'],3,7,Conservative,Elections & Campaign Finance,"[['here', 's', 'an', 'ethical', 'senator_mccai...","[{'sent_idx': 1, 'word_idx': [4, 9, 25, 36], '..."
3,['Grateful Teen Gets Her Mom A Vaccination Tha...,['One teen proved that showing gratitude is as...,['One teen proved that showing gratitude is as...,"['One', 'teen', 'proved', 'that', 'showing', '...",[['Reporter']],['2019-03-13 14:04:27'],['Huffington Post'],['mandatory vaccinations'],1,11,Liberal,Vaccincation & GMO,"[['one', 'teen', 'proved', 'that', 'showing', ...","[{'sent_idx': 1, 'word_idx': [4, 8, 12], 'orig..."
4,['Governor Moonbeam Says Trump&#8217;s Tweets ...,"['\n', 'RUSH: Here’s the former governor of Ca...",['RUSH: Here’s the former governor of Californ...,"['RUSH', ':', 'Here', '’', 's', 'the', 'former...",[['George Prayias']],['2019-01-24 13:30:00'],['rushlimbaugh.com'],['abortion'],3,1,Conservative,Social Issues,"[['rush', 'here', 's', 'the', 'former', 'mitt_...","[{'sent_idx': 1, 'word_idx': [7, 14, 15, 18, 1..."


In [199]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import re

bigram = Phraser.load("./bigram.model")

def preprocess_text(text):
    if type(text) == float:
        print(text)
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

def cascade_html(row):
    positions = {}
    
    for position_dict in ast.literal_eval(row['cands']):
        target_sent_idx = position_dict['sent_idx']
        target_word_idx = position_dict['word_idx']
        positions[target_sent_idx] = target_word_idx
        
    print_result = """<div>"""
    
    for sent_idx, sent in enumerate(ast.literal_eval(row['paras'])):
        words = preprocess_text(sent).split()
        words = bigram[words]
        for word_idx, word in enumerate(words):
#             print(sent_idx)
#             print(positions)
            if sent_idx in positions.keys():
                if word_idx in positions[sent_idx]:
                    print_result += ' '.join([' ', '<span style="background:#feffb8;">', word, '</span>'])
                else:
                    print_result = print_result + ' ' + word
            else:
                print_result = print_result + ' ' + word
        print_result = print_result + '\n\n'
    print_result = print_result + "</div>"
    return print_result     

In [300]:
from ipywidgets import AppLayout, Button, Layout
from IPython.display import display
from ipywidgets import TwoByTwoLayout
from ipywidgets import Layout, Button, Box

def create_expanded_button(description, button_style, icon):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'), icon=icon)

i = 0


def create_html(flag):
    global i
    print('current html i', i)
    if flag:
        i += 1
        idx = min(i+1, len(test_data_sample_cpy) - 1)
    else:
        i -= 1
        idx = max(i-1, 0)
    html = (cascade_html(test_data_sample_cpy.iloc[idx]))

    
    return widgets.HTML(
        value=html,
    )

previous_button = create_expanded_button("Previous", 'danger', 'arrow-left')
next_button = create_expanded_button("Next", 'info', 'arrow-right')
save_button = create_expanded_button("Save", 'success', 'check')

footer = TwoByTwoLayout(height='80px', width="70%",
                top_left=previous_button,
                bottom_left=next_button,
                bottom_right=save_button)

out = widgets.Output(layout={'border': '1px solid black'})


items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column', 
                    align_items='stretch')

def create_drop_down(options, target):
    return widgets.Dropdown(
        options = options,
        description=target,
    )


def create_drop_down_list(drop_flag):
    global i
    print('current dropdown i', i)
    if drop_flag:
        i += 1
        idx = min(i+1, len(test_data_sample_cpy) - 1)
    else:
        i -= 1
        idx = max(i-1, 0)
    items = []
    for cands_dict in ast.literal_eval(test_data_sample_cpy.iloc[i]['cands']):
        thelist = np.array(cands_dict['cands_result'])[:, 0].tolist()
        options=[item for item in zip(thelist, range(len(thelist)))]
        items.append(create_drop_down(options, thelist[0]))
#         items.append(create_drop_down(options, cands_dict['original_word']))
    box = Box(children=items, layout=box_layout)
    return box


def on_save_button_clicked(b):
    
    out.append_stdout('Next')

save_button.on_click(on_save_button_clicked)

def on_next_button_clicked(b):
    print("next button clicked!")
    new_box = create_drop_down_list(1)
    new_html_wid = create_html(1)
    show_UI(new_html_wid, new_box, footer)
    out.append_stdout('Next')

next_button.on_click(on_next_button_clicked)

def on_previous_button_clicked(b):
    print("previous button clicked!")
    new_html_wid = create_html(0)
    new_box = create_drop_down_list(0)
    show_UI(new_html_wid, new_box, footer)
    out.append_stdout('Previous')

previous_button.on_click(on_previous_button_clicked)

def show_UI(text_content, box, footer):
    out.clear_output()
    new_html_wid = create_html(1)
    new_box = create_drop_down_list(1)
    with out:
        display(AppLayout(header=None,
              pane_widths=[2, 2, 2],
              left_sidebar=None,
              center=new_html_wid,
              right_sidebar=new_box,
              footer=footer))

show_UI(text_content, box, footer)
out

current html i 0
current dropdown i 1


Output(layout=Layout(border='1px solid black'))

In [285]:
from ipywidgets import Layout, Button, Box

items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column', 
                    align_items='stretch')

words = ['correct', 'horse', 'battery', 'staple']

def create_drop_down(options, target):
    return widgets.Dropdown(
#         options=[item for item in zip(thelist, range(len(thelist)))],
        options = options,
#         value=2,
#         description=dist['original_word'],
        description=target,
    )

current_value = []

def on_change(change):
    global current_value
    if change['type'] == 'change' and change['name'] == 'value':
#         current_value[idx] = change['new']
        print(current_value)
        print("changed to %s" % change['new'])

def create_drop_down_list(row):
    items = []
    current_value = []
    for cands_dict in ast.literal_eval(row['cands']):
#         print(cands_dict)
        thelist = np.array(cands_dict['cands_result'])[:, 0].tolist()
        options=[item for item in zip(thelist, range(len(thelist)))]
#         print(options)
#         print(cands_dict['original_word'])
        items.append(create_drop_down(options, cands_dict['original_word']))
    for idx, btn in enumerate(items):
        current_value.append(btn.value)
        btn.observe(on_change)
    box = Box(children=items, layout=box_layout)
    return box

create_drop_down_list(test_data_sample_cpy.iloc[1])

Box(children=(Dropdown(description='amid', options=(('despite', 0), ('amid', 1)), value=0), Dropdown(descripti…